# Perform Azimuthal Integration 

## save in format for xrd_fit

In [1]:
from nexusformat.nexus import *
import pyFAI, fabio, h5py
import numpy as np
import glob, os
from ipywidgets import Button, Layout, Dropdown
import itertools
import csv
import logging 
import math
import pathlib
import warnings

In [2]:
# Define paths
data_dir = r'Z:\Diamond_MG28894-1_DATA BACKUP\rawdata'
cal_dir = r"Z:\Diamond_MG28894-1_DATA BACKUP\processing"
ponifile_1050mm = cal_dir + "\MG28894-1_poni1_1050.poni"
ponifile_2350mm = cal_dir + "\MG28894-1_poni2_2350.poni"
ponifile_750mm = cal_dir + "\MG28894-1_poni3_750.poni"

In [3]:
# List the directories in the root directory, newest first

dir_list=[name.split('\\')[-1].split(".")[0] for name in sorted(glob.glob(data_dir + '\*.nxs'))]
drop_down = Dropdown(options=dir_list, value=dir_list[0], description='scanID:', disabled=False, 
                             layout=Layout(width='40%'), style={'description_width': 'initial'})

def dropdown_handler(change):
    global scanID
    scanID = change.new
drop_down.observe(dropdown_handler, names='value')
display(drop_down)

IndexError: list index out of range

In [29]:
# Save diffraction data in required format: intensity \t C0 \t C1 ... \t Cn

####### make function that performs the azimuthal integration and returns 2th, d, I & metadata (and polarization factor)
def cake_data(scan_num, fpath, frame_format, metadata_path, metadata_format, pyFAI_ponyfile, wavelength=0.13865686822921588, azi_1=85, azi_range=10, r_bins=1000, partial_rings=False):
    """
    Performs azimuthal integration of opposing "slices" between the two defined azimuths (azi_1 & azi_2) for the 
    frame (scan_num)
    Returns data in dictionary
    *Polarization factor should be checked
    """
    
    #filepath & 
    filename = fpath + str(scan_num).zfill(5) + frame_format
    data = fabio.open(filename)
    
    #polarization factor correction for vertical and horiz correction
    av_azimuth = (azi_1 + azi_range) / 2
    p_factor = np.sin(np.radians(2*av_azimuth-90))
    
    x, y = pyFAI_ponyfile.integrate1d(data=data.data, npt=4000, polarization_factor=p_factor,
                  azimuth_range = (azi_1, azi_1+azi_range), radial_range=(2.5, 6.5),
                    method='bbox_cython', unit='2th_deg')
  
    d = wavelength/(2*np.sin(np.deg2rad(x/2)))

    
    fname = metadata_path + metadata_format
    metadata = nxload(fname)

    load = metadata.entry1.instrument.adc01_06.adc01_06
    pos = metadata.entry1.instrument.adc01_07.adc01_07
    temp = metadata.entry1.instrument.adc01_08.adc01_08
    
    metadata = {"load": load*200,
               "position": pos*0.5,
               "temp": temp*150}
    
    return {'2theta_deg':x, 'd_A':d, 'intensity':y, 'metadata':metadata, 'p_factor':p_factor}


In [30]:
fpath = data_dir + "\\" + scanID + "-pilatus2M-files" + "\\" 
frame_format = ".tif"
metadata_path = data_dir + "\\" + scanID
metadata_format = ".nxs"
ai = pyFAI.load(ponifile_1050mm)

In [31]:
# generate list of frames ---------------------------------
frames = glob.glob(data_dir + "\\" + scanID + "-pilatus2M-files" + "\\" + "*")
frames = np.array(sorted([int(name.split('\\')[-1].split(".")[0]) for name in frames]))
# ----------------------------------------------------------------------------

# Setup list of azimuths to integrate (0 is EAST; range(-180, 180); this is a pain in the arse)
azimuths = list(np.arange(0, 180, 5)) + list(np.arange(-180, 0, 5)) # opposing slices are combined in def(cake_data)
print([azi for azi in azimuths], len(azimuths))

##### setup test list ##### delete this once working
frames = list(range(1,2))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, -180, -175, -170, -165, -160, -155, -150, -145, -140, -135, -130, -125, -120, -115, -110, -105, -100, -95, -90, -85, -80, -75, -70, -65, -60, -55, -50, -45, -40, -35, -30, -25, -20, -15, -10, -5] 72


In [32]:
######### cake *ALL* data and add to text file ##########

######## processing heavy DO NOT RUN UNLESS SURE #########

for frame in frames:
    # extract two theta values
    data = cake_data(frame,
                 fpath, 
                 frame_format, 
                 metadata_path, 
                 metadata_format, 
                 ai, 
                 azi_1=azimuths[0],
                 azi_range=10, 
                 wavelength=0.13865686822921588,
                 r_bins=4000,
                 partial_rings=False) 
    data = [list(data['2theta_deg'])]

    ##### now append all other cake slices ####
    for az in azimuths:
        i_data = cake_data(frame, fpath,frame_format, metadata_path, metadata_format, ai, azi_1=az,azi_range=10, 
                           wavelength=0.13865686822921588, r_bins=4000, partial_rings=False) 
        data.append(list(i_data["intensity"]))
    
    # transpose into column format
    T_data = list(map(list, zip(*data)))
    
    # write to file
    fname = "Integrated//" + "Integrated_" + str(scanID) + '//' + f'{frame:05d}' + ".txt"
    with open(fname, 'w+', newline='') as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerows(T_data)

In [86]:
# supress warnings when TIFFs are read
logging.getLogger("fabio.TiffIO").setLevel(logging.ERROR)

# user inputs
number_of_points = 4000
number_of_cakes = 36

# rotate the detector so that the cardinal direction is in the center of the first cake.
first_cake_angle = 360 / number_of_cakes
ai.rot3 = (first_cake_angle / 2) * (math.pi / 180) # convert rotation to radians

# get a list of the files
image_list = sorted(pathlib.Path(fpath).glob("*.tif"))

In [87]:
for image_path in image_list[0:1]:
    # create empty array
    caked_data = np.zeros((number_of_cakes + 1, number_of_points))
    
    # create an image array and cake the data
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        image = fabio.open(image_path)
    pattern_image_array = image.data
    result2d = ai.integrate2d(pattern_image_array,
                            number_of_points,
                            number_of_cakes,
                            radial_range = (2.5, 6.5),
                            unit="2th_deg",
                            polarization_factor=0.99,
                            method='full_csr')
    
    # flip the intensity data to order cakes clockwise rather than anticlockwise
    intensity = np.flip(result2d.intensity.T, axis=1)
    
    # reshape radial labels to 2D array so they can be attached to the intensity data.
    radial = np.reshape(result2d.radial, (-1, 1))
    
    result_array = np.hstack((radial, intensity))
    
    # write out the caked data to a text file
    output_path = r"Z:/Diamond_MG28894-1_DATA_BACKUP/processing/Integrated/" + scanID + "/" + f"{image_path.stem}.txt"
    
    np.savetxt(output_path, result_array, delimiter='\t')
    
    